In [10]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse

from pydantic import BaseModel, AfterValidator, BeforeValidator, field_serializer, PlainSerializer

In [2]:
def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value


DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [3]:
class Model(BaseModel):
    dt: DateTimeUTC

In [8]:
def dt_json_serializer(dt: datetime) -> str:
    return dt.strftime('%Y/%m/%d %I:%M:%p (UTC)')

In [14]:
class Model(BaseModel):
    dt: DateTimeUTC

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_dt_to_json(self, dt: datetime) -> str:
        return dt_json_serializer(dt)

In [17]:
m = Model(dt="2020/1/1 3pm")

In [18]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00:PM (UTC)"}'

In [19]:
DateTimeUTC = Annotated[
    datetime,
    BeforeValidator(parse_datetime),
    AfterValidator(make_utc),
    PlainSerializer(dt_json_serializer, when_used="json-unless-none"),
]


In [20]:
class Model(BaseModel):
    dt: DateTimeUTC

In [23]:
m = Model(dt="2020/1/1 3pm")

In [24]:
m

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [22]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00:PM (UTC)"}'